In [1]:
import sys
sys.path.append('/kaggle/input/cags-competition')
#!/usr/bin/env python3
import argparse
import datetime
import os
import re
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "2")  # Report only TF errors by default

import numpy as np
import tensorflow as tf
import time

from cags_dataset import CAGS
import efficient_net


# TODO: Define reasonable defaults and optionally more parameters
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=16, type=int, help="Batch size.")
parser.add_argument("--epochs", default=80, type=int, help="Number of epochs.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--threads", default=64, type=int, help="Maximum number of threads to use.")
##
parser.add_argument("--dropout", default=0.3, type=float, help="Dropout regularization.")
parser.add_argument("--decay_steps", default=100, type=int, help="decay_steps.")
parser.add_argument("--learning_rate", default=0.05, type=float, help="Initial learning rate.")
parser.add_argument("--learning_rate_final", default=0.005, type=float, help="Final learning rate.")
parser.add_argument("--filter_sizes", default=[2, 16, 32, 64, 128],  nargs="*", type=int, help="Filters for deconv-")
parser.add_argument("--momentum", default=0.85, type=float, help="Momentum.")
parser.add_argument("--optimizer", default="SGD", type=str, help="Optimizer to use.")
args = parser.parse_args([] if "__file__" not in globals() else None)


def main(args: argparse.Namespace) -> None:
    # Fix random seeds and threads
    # tf.keras.utils.set_random_seed(args.seed)
    tf.config.threading.set_inter_op_parallelism_threads(args.threads)
    tf.config.threading.set_intra_op_parallelism_threads(args.threads)

    # Create logdir name
    args.logdir = os.path.join("logs", "{}-{}-{}".format(
        os.path.basename(globals().get("__file__", "notebook")),
        datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S"),
        ",".join(("{}={}".format(re.sub("(.)[^_]*_?", r"\1", k), v) for k, v in sorted(vars(args).items())))
    ))

    # Load the data
    cags = CAGS()

    generator = tf.random.Generator.from_seed(args.seed)
    def train_augment(image: tf.Tensor, label: tf.Tensor): # -> Tuple[tf.Tensor, tf.Tensor]:
        if generator.uniform([]) >= 0.5:
            image = tf.image.flip_left_right(image)
        image = tf.image.resize_with_crop_or_pad(image, CAGS.H + 6, CAGS.W + 6)
        image = tf.image.resize(image, [generator.uniform([], CAGS.H, CAGS.H + 12 + 1, dtype=tf.int32),
                                        generator.uniform([], CAGS.W, CAGS.W + 12 + 1, dtype=tf.int32)])
        image = tf.image.crop_to_bounding_box(
            image, target_height=CAGS.H, target_width=CAGS.W,
            offset_height=generator.uniform([], maxval=tf.shape(image)[0] - CAGS.H + 1, dtype=tf.int32),
            offset_width=generator.uniform([], maxval=tf.shape(image)[1] - CAGS.W + 1, dtype=tf.int32),
        )
        return image, label

    # Load the EfficientNet-B0 model
    efficientnet_b0 = efficient_net.pretrained_efficientnet_b0(include_top=False)

    # TODO: Create the model and train it
    # building model layers
    inputs = tf.keras.layers.Input(shape=[CAGS.H, CAGS.W, CAGS.C])
    hidden  = efficientnet_b0(inputs, training=True)
    out, c5, c4, c3, c2, c1  = hidden

    cs = [c4, c3, c2, c1]
    out = c5

    # # deconvolutions
    for c, filter_size in zip(cs, args.filter_sizes[:4]):
      out = tf.keras.layers.Conv2DTranspose(2, (3, 3), strides=(2, 2), padding="same")(out)
      out = tf.keras.layers.Concatenate()([out, c])
      out = tf.keras.layers.BatchNormalization()(out)
      out = tf.keras.layers.Dropout(args.dropout)(out)
              
    
    # deconv. wo concat
    out = tf.keras.layers.Conv2DTranspose(args.filter_sizes[4], (3, 3), strides=(2, 2), padding="same")(out)
    out = tf.keras.layers.BatchNormalization()(out)
    out = tf.keras.layers.Dropout(args.dropout)(out)

    # # conv outputs
    outputs = tf.keras.layers.Conv2D(1, (1,1), padding="same", activation="sigmoid")(out)
    model    =  tf.keras.Model(inputs=inputs, outputs=outputs)

    # choosing lr
    learning_rate = tf.keras.optimizers.schedules.PolynomialDecay(  args.learning_rate, args.decay_steps, 
                                                                    end_learning_rate=args.learning_rate_final, power=1.0,
                                                                    cycle=False, name='linear_lr_decay')

    # # choosing optimizer
    if args.optimizer=='SGD':
        if args.momentum != None:
            momentum = args.momentum
        else:
            momentum = 0.0
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum, nesterov=False, name='SGD')

    if args.optimizer=='Adam':
        optimizer =  tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam')



    model.compile(
        optimizer=optimizer,
        loss=tf.losses.BinaryCrossentropy(),
        # metrics=[tf.metrics.SparseCategoricalAccuracy("accuracy")],
        metrics=[cags.MaskIoUMetric("IoU")],
        )
    
    train = cags.train.map(lambda example: (example["image"], example["mask"]) )
    # train = train.map(train_augment)
    train = train.shuffle(10000, seed=args.seed)
    train = train.batch(args.batch_size)

    dev = cags.dev.map( lambda example: (example["image"], example["mask"]) )  
    dev = dev.shuffle(10000, seed=args.seed)
    dev = dev.batch(args.batch_size)

    # fit
    tb_callback = tf.keras.callbacks.TensorBoard(args.logdir, update_freq=100, profile_batch=0)
    logs = model.fit(
            # cags.train.map(CAGS.parse),
            train,
            batch_size=args.batch_size, epochs=args.epochs,
            validation_data=dev,
            callbacks=[tb_callback]
        )
    

    # Generate test set annotations, but in `args.logdir` to allow parallel execution.
    os.makedirs(args.logdir, exist_ok=True)
    with open(os.path.join(args.logdir, "cags_segmentation.txt"), "w", encoding="utf-8") as predictions_file:
        # TODO: Predict the masks on the test set
        # test = cags.test.map(lambda example: (example["image"]))
        test = cags.test.map(lambda example: (example["image"], example["mask"]))
        test = test.batch(args.batch_size)
        test_masks = model.predict(test)
        

        for mask in test_masks:
            zeros, ones, runs = 0, 0, []
            for pixel in np.reshape(mask >= 0.5, [-1]):
                if pixel:
                    if zeros or (not zeros and not ones):
                        runs.append(zeros)
                        zeros = 0
                    ones += 1
                else:
                    if ones:
                        runs.append(ones)
                        ones = 0
                    zeros += 1
            runs.append(zeros + ones)
            print(*runs, file=predictions_file)



In [3]:
# time.sleep(360)

In [5]:
# TODO: Define reasonable defaults and optionally more parameters
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=16, type=int, help="Batch size.")
parser.add_argument("--epochs", default=250, type=int, help="Number of epochs.")
parser.add_argument("--seed", default=42, type=int, help="Random seed.")
parser.add_argument("--threads", default=64, type=int, help="Maximum number of threads to use.")
##
parser.add_argument("--dropout", default=0.3, type=float, help="Dropout regularization.")
parser.add_argument("--decay_steps", default=1000, type=int, help="decay_steps.")
parser.add_argument("--learning_rate", default=0.03, type=float, help="Initial learning rate.")
parser.add_argument("--learning_rate_final", default=0.003, type=float, help="Final learning rate.")
parser.add_argument("--filter_sizes", default=[16, 40, 80, 112, 192],  nargs="*", type=int, help="Filters for deconv-")
parser.add_argument("--momentum", default=0.85, type=float, help="Momentum.")
parser.add_argument("--optimizer", default="SGD", type=str, help="Optimizer to use.")
args = parser.parse_args([] if "__file__" not in globals() else None)
print(args)
main(args)

Namespace(batch_size=16, decay_steps=1000, dropout=0.3, epochs=250, filter_sizes=[16, 40, 80, 112, 192], learning_rate=0.03, learning_rate_final=0.003, momentum=0.85, optimizer='SGD', seed=42, threads=64)
Epoch 1/250
134/134 [==============================] - 62s 384ms/step - loss: 0.6501 - IoU: 0.2802 - val_loss: 0.5884 - val_IoU: 0.2459
Epoch 2/250
134/134 [==============================] - 49s 359ms/step - loss: 0.5435 - IoU: 0.4244 - val_loss: 0.4826 - val_IoU: 0.5145
Epoch 3/250
134/134 [==============================] - 49s 360ms/step - loss: 0.4791 - IoU: 0.5156 - val_loss: 0.4005 - val_IoU: 0.6264
Epoch 4/250
134/134 [==============================] - 49s 359ms/step - loss: 0.4073 - IoU: 0.6029 - val_loss: 0.2741 - val_IoU: 0.7511
Epoch 5/250
134/134 [==============================] - 50s 363ms/step - loss: 0.3191 - IoU: 0.6961 - val_loss: 0.2132 - val_IoU: 0.8033
Epoch 6/250
134/134 [==============================] - 49s 359ms/step - loss: 0.2786 - IoU: 0.7369 - val_loss: 0.19

In [ ]:
time.sleep(36000)